In [1]:
## Mapper (FermionicOp to SparsePauliOp transformation) ## 
from qiskit_nature.second_q.mappers import JordanWignerMapper
qubit_mapper = JordanWignerMapper()

## For He6
num_orbitals = (6,6)
num_particles = (4,2)
num_spin_orbitals = 12
num_spatial_orbitals = int(num_spin_orbitals/2)
preserve_spin = True

def custom_excite(num_spatial_orbitals: int,num_particles:tuple):
    excite_list_dum =  [((0, 1), (4, 5)),((2, 3), (4, 5)),((6, 7), (8, 9)),((6, 7), (10, 11))]
    return excite_list_dum
## Define excitation 
vqe_excitations = custom_excite

## Reference state / Initial state ## 
from np_hartree_fock import * ## Import custom init state
initial_state = HartreeFock(
    num_orbitals = num_orbitals,
    num_particles = num_particles,
    qubit_mapper = qubit_mapper)

## Ansatz libraries ##
from qiskit_nature.second_q.circuit.library.ansatzes import UCC
# from ucc_trott import UCC as UCC_trott
## Ansatz
reps=1
var_form = UCC(
    num_particles=num_particles,
    num_spatial_orbitals=num_spatial_orbitals,
    excitations=vqe_excitations,
    qubit_mapper=qubit_mapper,
    reps=reps,
    initial_state=initial_state,
    preserve_spin= preserve_spin)


from qiskit_aer.backends import AerSimulator
from qiskit.primitives import BackendEstimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
shots = 8192
from FakeBackends.fake_johor_FJ001 import FakeJohorV2 as FJ001
backend_fj001 = FJ001(); backend_fj001_gpu = AerSimulator.from_backend(backend_fj001, method="automatic", device="GPU")
pass_man_fj001 = generate_preset_pass_manager(3, backend_fj001)
esti_gpu_fj001= BackendEstimator(backend=backend_fj001_gpu, options={"shots":shots}, bound_pass_manager = pass_man_fj001)
from numpy.random import rand


In [3]:
param = rand(4)*np.pi
print(param)
fin_cir = pass_man_fj001.run(var_form.assign_parameters(param))

print(fin_cir.decompose().decompose().decompose().depth())
print(fin_cir.count_ops())

[1.4652987  1.4002428  0.50561367 3.02479441]
250
OrderedDict([('rz', 208), ('cx', 155), ('sx', 140), ('x', 2)])


In [5]:
## The Hamiltonian
### Use the defined obs_twobody_df to construct the Hamiltonian
from qiskit_nature.second_q.operators import FermionicOp
import pandas as pd
import numpy as np
from qiskit.quantum_info.operators import SparsePauliOp

## Data Loading
obs_onebody_csv = "../Data/2121_He6_herm_1/2121_He6_herm_1-1B_H_input.csv"
obs_twobody_csv = "../Data/2121_He6_herm_1/2121_He6_herm_1-2B_H_input.csv"
obs_onebody_df = pd.read_csv(obs_onebody_csv)
obs_twobody_df = pd.read_csv(obs_twobody_csv)

### One body Terms: Single particle energy levels
tmp_ham_one = {}
for index, row in obs_onebody_df.iterrows():
    fina = int(row['i']); init = int(row['j'])
    the_onestring = "+_" +str(fina) + " " + "-_" +str(init) 
    tmp_ham_one[the_onestring] = row["epsilon"]
    tmp_ham = tmp_ham_one
    
### Two body Terms: Pairing interaction
tmp_ham_two = {}
for index, row in obs_twobody_df.iterrows():
    fina_1 = int(row['i']); fina_2 = int(row['j']); 
    init_1 = int(row['k']); init_2 = int(row['l']); 
    the_twostring = "+_" +str(fina_1) + " " + "+_" +str(fina_2) + " " + "-_" +str(init_1) + " " + "-_" +str(init_2)
    tmp_ham_two[the_twostring] = 0.25*row['V_ijkl']
    tmp_ham = tmp_ham_two

tmp_ham = {**tmp_ham_one, **tmp_ham_two}

## The Hamiltonian Fermionic operator are given by
Hamiltonian = FermionicOp(tmp_ham, 
                          num_spin_orbitals=num_spin_orbitals, 
                          copy=False)
hermitian_info = Hamiltonian.is_hermitian()

## Prepping Hamiltonian to be computed. Mapping. ## 
Hamiltonian_fermop_len = len(Hamiltonian)
Hamiltonian = qubit_mapper.map(Hamiltonian)

# Define observable to minimize (obs_to_minimize)
## Add a constant term to the Hamiltonian
sp_energies = obs_onebody_df["epsilon"].to_list()
E_sum_sp = sum(sp_energies[0:num_particles[0]] + sp_energies[num_orbitals[0]:num_orbitals[0]+num_particles[1]])
Identity_string = "I"*num_spin_orbitals
constant_term = SparsePauliOp(Identity_string, coeffs=np.array([-round(E_sum_sp,6)]))
obs_to_minimize = SparsePauliOp.sum([Hamiltonian, constant_term])


In [6]:
print("length of obs:", len(obs_to_minimize))
obs_to_minimize_GC = obs_to_minimize.group_commuting(qubit_wise=True)
print(len(obs_to_minimize_GC))

length of obs: 68
13
